Установка необходимых библиотек

In [1]:
!pip install pymorphy3
!pip install joblib
!pip install natasha

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 85.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=2fbc3c20d656380e04ade12a8e2c4599737741a7086e57ec9082961281552aa4
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26096 sha256=da280b4f2a14e7d1050c8941e5051041518809e3bf137539918420acd4bc4cf4
  Stored in 

Загрузка библиотек

In [2]:
import pandas as pd
import numpy as np
from pymorphy3 import MorphAnalyzer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    f1_score,
    classification_report,
)


from sklearn.model_selection import train_test_split
from collections import Counter
import json
import pickle
import joblib
import string
import re

Установка данных

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Константы

In [4]:
MAX_WORDS = 10000
RANDOM_STATE = 42

punctuation_marks = ('!', ',', '(', ')', ':', '-', '+', '?', '.', '«', '»', ';', '–', '-', '…', '—',
                      '#', '&', '%', '*', '“', '”', '`', "'", '"', '0', '1', '2', '3', '4', '5', '6', '7', '8',
                      '9', 's', 'x', 'l', '\\', '/') + tuple(string.ascii_lowercase)

# С помощью лемматизации, стематизации, pos-тегов нужно убрать из словаря и обработки этот снизу мусор
stop_words = stopwords.words("russian") + ['это', 'аж', 'ка', 'мя', 'ул', 'ом', 'ма', 'ый', 'мр', 'тр', 'чей',
                                           'сгс', 'ля', 'эппл', 'юсб', 'тлф', 'ася', 'азс', 'ик', 'нуть', 'юрий',
                                           'тат', 'эл', 'ух', 'джи', 'дог', 'грн', 'ия', 'изз', 'самс', 'дак', 'пвз',
                                           'ке', 'ин', 'гпс', 'сап', 'аля', 'хны', 'эх', 'зы']
stop_words.remove('не')

morph = MorphAnalyzer()

Файлы

In [9]:
# Файл для обучения
FILE_NAME = 'БЕЗ ТУПЫХ ЗНАКОВ - 10'
reviews = pd.read_csv(f'{FILE_NAME}.csv', sep=',')
reviews.head(5)

,Unnamed: 0,Text,Score
0,0,Лучшая детская парикмахерская! А чего стоят си...,2
1,1,"Ребята, советую брать если нужен телефон на дв...",2
2,2,Единственный минус что нет фуд кортов и мало м...,2
3,3,"Красивый храм. Расположен правда не на "" ходу""...",2
4,4,"Очень люблю этот супермаркет, всегда чисто, хо...",2


Переиндексация

In [10]:
reviews.reset_index(drop=True)

,Unnamed: 0,Text,Score
0,0,Лучшая детская парикмахерская! А чего стоят си...,2
1,1,"Ребята, советую брать если нужен телефон на дв...",2
2,2,Единственный минус что нет фуд кортов и мало м...,2
3,3,"Красивый храм. Расположен правда не на "" ходу""...",2
4,4,"Очень люблю этот супермаркет, всегда чисто, хо...",2
...,...,...,...
9995,9995,"Сенсор ужасный - уйма ложных срабатываний, вме...",1
9996,9996,Атмосферное место с нормальными ребятами! Гост...,0
9997,9997,"Радует то, что я его купил за 16500, а не за 2...",1
9998,9998,Нужен был ТЕЛЕФОН(для всего остального есть iP...,2


Удаление пустых столбцов

In [11]:
reviews = reviews[['Text', 'Score']]
reviews

,Text,Score
0,Лучшая детская парикмахерская! А чего стоят си...,2
1,"Ребята, советую брать если нужен телефон на дв...",2
2,Единственный минус что нет фуд кортов и мало м...,2
3,"Красивый храм. Расположен правда не на "" ходу""...",2
4,"Очень люблю этот супермаркет, всегда чисто, хо...",2
...,...,...
9995,"Сенсор ужасный - уйма ложных срабатываний, вме...",1
9996,Атмосферное место с нормальными ребятами! Гост...,0
9997,"Радует то, что я его купил за 16500, а не за 2...",1
9998,Нужен был ТЕЛЕФОН(для всего остального есть iP...,2


Удаление NaN, повторяющихся текстов

In [12]:
reviews = reviews.dropna(subset=['Text', 'Score'])

In [13]:
reviews = reviews.drop_duplicates('Text')

Функции

In [14]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # эмодзи эмоций
                           u"\U0001F300-\U0001F5FF"  # символы и пиктограммы
                           u"\U0001F680-\U0001F6FF"  # эмодзи транспорта
                           u"\U0001F1E0-\U0001F1FF"  # эмодзи флагов стран
                           u"\U00002500-\U00002BEF"  # китайские иероглифы
                           u"\U00002702-\U000027B0"
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\U0001f926-\U0001f937"
                           u"\U00010000-\U0010ffff"
                           u"\u2640-\u2642"
                           u"\u2600-\u2B55"
                           u"\u200d"
                           u"\u23cf"
                           u"\u23e9"
                           u"\u231a"
                           u"\ufe0f"  # variation selector
                           u"\u3030"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


def preprocess(text, stop_words, punctuation_marks, morph):
    """Разбивка предложений на слова"""
    text = str(text).lower()
    # Удаление эмодзи
    text = remove_emoji(text)
    # Удаление пунктуации
    for letter in punctuation_marks:
        text = text.replace(letter, ' ')

    tokens = word_tokenize(text)
    preprocessed_text = []
    for token in tokens:
        if len(token) > 1:
            lemma = morph.parse(token)[0].normal_form
            if lemma not in stop_words:
                preprocessed_text.append(lemma)
    return preprocessed_text


def text_to_sequence(txt, word_to_index):
    seq = []
    for word in txt:
        index = word_to_index.get(word, 1) # 1 означает неизвестное слово
        # Неизвестные слова не добавляем в выходную последовательность
        if index != 1:
            seq.append(index)
    return seq


def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        for index in sequence:
            results[i, index] += 1.
    return results

###Обработка текста

In [15]:
reviews['Preprocessed_texts'] = reviews.apply(lambda row: preprocess(row['Text'], stop_words, punctuation_marks, morph), axis=1)

### Запуск готового словаря популярных слов

In [19]:
# Cловарь самых популярных слов
with open("dictionary4.json", "r") as f:
    word_to_index = json.load(f)

### Создание готового словаря популярных слов

In [18]:
# Считаем частоту слов во всех отзывах
words = Counter()

for txt in reviews['Preprocessed_texts']:
    words.update(txt)


# Словарь, отображающий слова в коды
word_to_index = dict()
# Словарь, отображающий коды в слова
index_to_word = dict()

#Создаем словари
for i, word in enumerate(words.most_common(MAX_WORDS - 2)):
    word_to_index[word[0]] = i + 2
    index_to_word[i + 2] = word[0]

### Сохранение словаря популярных слов

In [20]:
  with open("dictionary_new.json", "w") as f:
    json.dump(word_to_index, f)

### Обработка списка слов

In [21]:
reviews['Sequences'] = reviews.apply(lambda row: text_to_sequence(row['Preprocessed_texts'], word_to_index), axis=1)

Замена оценок на:
0 - отрицательный,
1 - нейтральный,
2 - позитивный.

In [22]:
mapping = {"1": 0, "2": 0, "3": 1, "4": 2, "5": 2}
reviews.replace({'Score': mapping}, inplace=True)

In [23]:
reviews

,Text,Score,Preprocessed_texts,Sequences
0,Лучшая детская парикмахерская! А чего стоят си...,2,"[хороший, детский, парикмахерская, стоять, сид...","[6, 239, 3690, 464, 3050, 14, 3152, 1463, 330,..."
1,"Ребята, советую брать если нужен телефон на дв...",2,"[ребята, советовать, брать, нужный, телефон, с...","[262, 54, 33, 77, 4, 279, 254, 54, 935, 1168, ..."
2,Единственный минус что нет фуд кортов и мало м...,2,"[единственный, минус, фуд, корт, мало, магазин]","[213, 96, 3381, 3687, 229, 24]"
3,"Красивый храм. Расположен правда не на "" ходу""...",2,"[красивый, храм, расположить, правда, не, ход,...","[87, 1705, 481, 257, 2, 1651, 1864, 294, 3439,..."
4,"Очень люблю этот супермаркет, всегда чисто, хо...",2,"[очень, любить, супермаркет, чисто, хороший, в...","[3, 269, 2557, 98, 6, 44, 163, 169, 2, 780, 2,..."
...,...,...,...,...
9995,"Сенсор ужасный - уйма ложных срабатываний, вме...",1,"[сенсор, ужасный, уйма, ложный, срабатывание, ...","[404, 419, 5845, 5649, 5356, 750, 1175, 164, 4..."
9996,Атмосферное место с нормальными ребятами! Гост...,0,"[атмосферный, место, нормальный, ребята, госте...","[893, 18, 142, 262, 126, 54, 334, 93, 104, 11,..."
9997,"Радует то, что я его купил за 16500, а не за 2...",1,"[радовать, купить, не, тыс, коллега, работа, т...","[214, 20, 2, 656, 1694, 36, 4, 64, 153, 316, 2..."
9998,Нужен был ТЕЛЕФОН(для всего остального есть iP...,2,"[нужный, телефон, весь, остальной, звонить, ма...","[77, 4, 7, 193, 291, 1223, 2946, 95]"


### Сохранение файла

In [24]:
reviews.to_csv(f'{FILE_NAME}-Обработано.csv')

Разделение файла

In [25]:
train, test = train_test_split(reviews, test_size=0.2)

x_train_seq = train['Sequences']
y_train = train['Score']

x_test_seq = test['Sequences']
y_test = test['Score']

Векторизация данных

In [26]:
x_train = vectorize_sequences(x_train_seq, MAX_WORDS)
x_test = vectorize_sequences(x_test_seq, MAX_WORDS)

Обучение модели

In [27]:
model = RandomForestClassifier()
model.fit(x_train, y_train)

RandomForestClassifier()

In [28]:
joblib.dump(model, 'Random Forest.joblib')

['Random Forest.joblib']

Загрузка модели

In [ ]:
model1 = joblib.load('Logistic Regression.joblib')
model2 = joblib.load('Naive Bayes.joblib')

Проверка

In [29]:
text = 'Игра кажется устаревшей и неинтересной. Графика ужасна, и управление крайне неудобно. Не рекомендую'

preprocessed_text = preprocess(text, stop_words, punctuation_marks, morph)
seq = text_to_sequence(preprocessed_text, word_to_index)
bow = vectorize_sequences([seq], MAX_WORDS)

In [30]:
model.predict_proba(bow)

array([[0.67, 0.17, 0.16]])

In [ ]:
model1.predict_proba(bow)

array([[0.08114911, 0.19260082, 0.72625007]])

In [ ]:
model2.predict_proba(bow)

array([[0.25528571, 0.07471429, 0.67      ]])

In [31]:
y_pred = model.predict(x_test)

accuray = accuracy_score(y_pred, y_test)
f1 = f1_score(y_pred, y_test, average="weighted")

print("Accuracy:", accuray)
print("F1 Score:", f1)

Accuracy: 0.8244734202607823
F1 Score: 0.877397183099431
